# **Importing Necessary Libraries**

- `streamlit`: Used to create the web interface.
- `uuid`: Generates unique identifiers (for future use if needed).
- `os`: Provides functions to work with the operating system, used here for file path management.
- `bot` & `statistics_chatbot` modules: Importing custom modules for querying the chatbot and updating/displaying statistics.


In [1]:
import streamlit as st
from uuid import uuid4
import os
from bot import *
from statistics_chatbot import update_statistics, get_statistics_display  # Importing statistics module

ModuleNotFoundError: No module named 'streamlit'

# **Setting Page Configuration**

- Sets the title to `"Team4ChatBot"` and configures the layout to be wide, enhancing the app's visual presentation.


In [2]:
st.set_page_config(page_title="Team4ChatBot", layout="wide")
print("Page configuration set")


NameError: name 'st' is not defined

# **Applying Custom CSS Styling**

- Loads a custom CSS file for styling the app and applies it using Streamlit’s Markdown.
- The print statement confirms that the styling is applied.


In [ ]:
css_file_path = os.path.join(os.path.dirname(__file__), 'styles', 'styles.css')
with open(css_file_path) as f:
    st.markdown(f'<style>{f.read()}</style>', unsafe_allow_html=True)
print("CSS styling applied")


# **Setting the Page Title**

- Displays the main title for the chatbot application.


In [ ]:
st.title("Team4 Chatbot")
print("Page title set")


# **Sidebar Header for Static Report Metrics**

- Adds a header in the sidebar to display static metrics.


In [ ]:
st.sidebar.header("10 Statistics Report")
print("Sidebar header added")


# **Initializing Chat History**

- Ensures chat history is maintained across interactions by storing it in `st.session_state`.


In [ ]:
if 'chat_history' not in st.session_state:
    st.session_state.chat_history = []
print("Chat history session state initialized")


# **Setting Up Chat Input Box**

- Creates an input box where users can type messages.


In [ ]:
user_input = st.chat_input("Message writing assistant")
print("Chat input box displayed")


# **Fetching and Displaying Sidebar Statistics**

- Calls `get_statistics_display()` to retrieve statistical data and displays it in the sidebar.


In [ ]:
current_stats = get_statistics_display()
for key, value in current_stats.items():
    st.sidebar.markdown(f'<div class="question-box">{key}: {value}</div>', unsafe_allow_html=True)
print("Sidebar statistics displayed")


# **Initializing Milvus Vector Store**

- Loads and initializes Milvus, the vector store for document storage and retrieval, displaying a spinner while loading.


In [ ]:
if 'conversation' not in st.session_state:
    st.session_state['conversation'] = []
    with st.spinner("Initializing, Please Wait..."):
        vector_store = initialize_milvus()
print("Milvus vector store initialized")


# **Cleaning Up Repeated Text in the Bot's Response**

- **Purpose**: Removes repeated sentences from the bot’s response for clarity.
- **Logic**: Splits the response into sentences, checks for duplicates, and returns a cleaned version.
- **Print statement**: Indicates when cleaning is done.


In [ ]:
def clean_repeated_text(text):
    sentences = text.split('. ')
    seen = set()
    cleaned_sentences = []
    
    for sentence in sentences:
        if sentence not in seen:
            cleaned_sentences.append(sentence)
            seen.add(sentence)
    print("Repeated text cleaned")
    return '. '.join(cleaned_sentences)


# **Handling User Input**

- **Function Purpose**: Handles the main chatbot response cycle.
- **Steps**:
  - Starts a timer to measure response time.
  - Adds the user’s input to the chat history and displays it.
  - Invokes `query_rag` to get the bot’s response and cleans it using `clean_repeated_text`.
  - Updates chat history and displays the bot’s response.
  - Calculates and updates response statistics.


In [ ]:
def handle_user_input(user_input):
    import time
    start_time = time.time()
    print("User input received, starting response generation timer")

    st.session_state.chat_history.append({"role": "user", "content": user_input})
    st.markdown(f"<div class='user-message'>{user_input}</div>", unsafe_allow_html=True)
    print("User input added to chat history")

    with st.spinner("Response Generating, please wait..."):
        bot_response, citations = query_rag(user_input)
        cleaned_response = clean_repeated_text(bot_response)
        print("Bot response and citations generated and cleaned")

    full_response = cleaned_response
    if citations:
        full_response += f"\n\nReferences: {citations}"
    st.session_state.chat_history.append({"role": "bot", "content": full_response})
    print("Bot response added to chat history")

    response_time = time.time() - start_time
    correct_answer = True
    update_statistics(user_input, full_response, response_time, correct_answer)
    print(f"Response time calculated: {response_time}s")


# **Processing User Input When Available**

- Calls `handle_user_input` if there is a new user message.


In [ ]:
if user_input:
    handle_user_input(user_input)


# **Rendering Chat History**

- **Purpose**: Displays the complete chat history, showing only bot responses.


In [ ]:
for message in st.session_state.chat_history:
    if message['role'] == 'user':
        continue
    else:
        st.markdown(f"{message['content']}")
print("Chat history rendered")


# **Feedback Mechanism for User Satisfaction**

- **Feedback Feature**: Allows users to provide thumbs-up or thumbs-down feedback.
- **Mapping**: Uses icons to indicate user sentiment visually.


In [ ]:
sentiment_mapping = [":material/thumb_down:", ":material/thumb_up:"]
selected = st.feedback("thumbs")
if selected is not None:
    st.markdown(f"You selected: {sentiment_mapping[selected]}")
print("Feedback mechanism initialized")
